In [0]:

src_final_df = spark.read.table("bronze.udemy1")



In [0]:
# # display(src_final_df)
# dbutils.fs.rm("dbfs:/mini_project/DataLake/silver/udemy", True)
src_final_df.write.format("parquet").mode("overwrite").saveAsTable("silver.udemy2")



In [0]:
# src_final_df = spark.read.table("bronze.udemy1")

In [0]:
from pyspark.sql.functions import col
src_final_df = src_final_df.withColumn("Enrollment", col("Enrollment").cast("integer"))
src_final_df = src_final_df.dropna(how="any")

In [0]:
display(src_final_df)

index,Title,Summary,Enrollment,Stars,Rating,Link,category
0,The Complete SQL Bootcamp 2020: Go from Zero to Hero,Become an expert at SQL!,301243,4.7,79919,https://www.udemy.com/course/the-complete-sql-bootcamp/,bussiness
1,Tableau 2020 A-Z: Hands-On Tableau Training for Data Science,Learn Tableau 2020 for data science step by step. Real-life data analytics exercises & quizzes included. Learn by doing!,211674,4.6,55582,https://www.udemy.com/course/tableau10/,bussiness
2,PMP Exam Prep Seminar - PMBOK Guide 6,PMP Exam Prep Seminar - Earn 35 PDUs by completing the entire PMP course,157957,4.6,53858,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,bussiness
3,The Complete Financial Analyst Course 2020,"Excel, Accounting, Financial Statement Analysis, Business Analysis, Financial Math, PowerPoint: Everything is Included!",249097,4.5,47415,https://www.udemy.com/course/the-complete-financial-analyst-course/,bussiness
4,An Entire MBA in 1 Course:Award Winning Business School Prof,** #1 Best Selling Business Course! ** Everything You Need to Know About Business from Start-up to IPO,376913,4.5,42101,https://www.udemy.com/course/an-entire-mba-in-1-courseaward-winning-business-school-prof/,bussiness
5,Microsoft Power BI - A Complete Introduction [2020 EDITION],"Learn how to use Microsoft's Power BI Tools, including Power BI Desktop, Power BI Pro (Service) and PowerBI Developer",126880,4.6,38771,https://www.udemy.com/course/powerbi-complete-introduction/,bussiness
6,Agile Crash Course: Agile Project Management; Agile Delivery,Get Agile Certified & Learn about the key and most important concepts and tools of Agile Project Management (Scrum),98700,4.3,31276,https://www.udemy.com/course/agile-crash-course/,bussiness
7,Beginner to Pro in Excel: Financial Modeling and Valuation,Financial Modeling in Excel that would allow you to walk into a job and be a rockstar from day one!,128940,4.5,29111,https://www.udemy.com/course/beginner-to-pro-in-excel-financial-modeling-and-valuation/,bussiness
8,Become a Product Manager | Learn the Skills & Get the Job,"The most complete course available on Product Management. 13+ hours of videos, activities, interviews, & more",114114,4.5,27879,https://www.udemy.com/course/become-a-product-manager-learn-the-skills-get-a-job/,bussiness
9,The Business Intelligence Analyst Course 2020,"The skills you need to become a BI Analyst - Statistics, Database theory, SQL, Tableau – Everything is included",118019,4.5,24582,https://www.udemy.com/course/the-business-intelligence-analyst-course-2018/,bussiness


In [0]:
pip install beautifulsoup4

Python interpreter will be restarted.
  Using cached beautifulsoup4-4.11.2-py3-none-any.whl (129 kB)
  Using cached soupsieve-2.4-py3-none-any.whl (37 kB)
Python interpreter will be restarted.


In [0]:

import requests
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from bs4 import BeautifulSoup
import requests

def fetch_data_from_url(url):
    data = requests.get(url)
    try:
        array=[]
        s = BeautifulSoup(data.text, 'html.parser')
        instructor_tag=s.find_all('div',{"ud-text-md instructor--instructor__job-title--3iHjg"})[0].text.replace(","," ")
        written_rating=s.find_all('div',{'class':"ud-block-list-item-content"})[-3].text.replace(",","")
        total_students=s.find_all('div',{'class':"ud-block-list-item-content"})[-2].text.replace(",","")
        total_courses=s.find_all('div',{'class':"ud-block-list-item-content"})[-1].text.replace(",","")
        instructor_rating=s.find_all('div',{'class':"ud-block-list-item-content"})[-4].text.replace(",","")
        latest_st_enroll=s.find_all("div",{'class':"enrollment"})[0].text.replace(",","")
        array.append(instructor_tag)
        array.append(written_rating)
        array.append(total_students)
        array.append(latest_st_enroll)
        array.append(total_courses)
        array.append(instructor_rating)
        images = s.find_all('img')
        course_images=[]
        for image in images:
            image_url = image['src']
            course_images.append(image_url)
        
        array.append(course_images[1])
        elements=s.find_all('span')
        
        z=[]
        for i in elements:
            z.append(i.text.replace(",",""))
        e=[]
        z = list(filter(lambda y: y != '', z))
#         e.append(z[5])
        e.append(s.find_all('span',{'class':"ud-heading-sm star-rating-module--rating-number--2xeHu"})[0].text.replace(",",""))
        e.append(z[6])
#         e.append(z[9])
        e.append(s.find_all('a',{'class':'ud-btn ud-btn-large ud-btn-link ud-heading-md ud-text-sm ud-instructor-links'})[0].text.replace(",",""))
        e.append(s.find_all('div',{'class':"last-update-date"})[0].text)

        all_content=", ".join(e)
        
        array.append(all_content)
        
#         return array
    except IndexError:
           array=[None,None,None,None,None,None,None,None,None,None,None]
#         instructor_tag=None
#         written_rating = None
#         instructor_tag=None
#         total_students = None
#         total_courses=None
#         instructor_rating = None
#         array.append(instructor_tag)
#         array.append(written_rating)
#         array.append(total_students)
#         array.append(total_courses)
#         array.append(instructor_rating)
#         array.append(None)
    return array
  
# url='https://www.udemy.com/course/the-complete-sql-bootcamp/'
# print(fetch_data_from_url (url))   

fetch_data_udf = udf(fetch_data_from_url, StringType())
src_final_df = src_final_df.withColumn("data", fetch_data_udf(col("Link")))


In [0]:
# display(src_final_df)
src_final_df.write.format("parquet").mode("overwrite").saveAsTable("silver.udemy_raw")



In [0]:
# club_data=spark.read.option('header',True).csv("dbfs:/FileStore/udemy_mnt/export__2_.csv")

club_data=src_final_df

In [0]:
from pyspark.sql.functions import split, col

split_df = club_data.select("index","Title","Summary","Enrollment","Stars","Rating","Link","category", split(col("data"), ",").alias("data"))

# extract values from the array
result_df = split_df.select("index","Title","Summary","Enrollment","Stars","Rating","Link","category",
                            col("data").getItem(0).alias("instructor_tag"), 
                            col("data").getItem(1).alias("written_rating"),
                            col("data").getItem(2).alias("total_students_author"),
                            col("data").getItem(3).alias("latest_enroll"),
                            col("data").getItem(4).alias("total_courses"),
                            col("data").getItem(5).alias("instructor_rating"),
                            col("data").getItem(6).alias("img_url"),
                            col("data").getItem(7).alias("course_rating"),
                            col("data").getItem(8).alias("written_course_rating"),
                            col("data").getItem(9).alias("author_name"),
                            col("data").getItem(10).alias("last_modified")
                           )



In [0]:
display(result_df)
# result_df=result_df.drop('total_students')

index,Title,Summary,Enrollment,Stars,Rating,Link,category,instructor_tag,written_rating,total_students_author,latest_enroll,total_courses,instructor_rating,img_url,course_rating,written_course_rating,author_name,last_modified
0,The Complete SQL Bootcamp 2020: Go from Zero to Hero,Become an expert at SQL!,301243,4.7,79919,https://www.udemy.com/course/the-complete-sql-bootcamp/,bussiness,[Head of Data Science at Pierian Training,1013752 Reviews,3269886 Students,641558 students,60 Courses,4.6 Instructor Rating,https://img-c.udemycdn.com/course/240x135/762616_7693_3.jpg,4.7,(164866 ratings),Jose Portilla,Last updated 5/2022]
1,Tableau 2020 A-Z: Hands-On Tableau Training for Data Science,Learn Tableau 2020 for data science step by step. Real-life data analytics exercises & quizzes included. Learn by doing!,211674,4.6,55582,https://www.udemy.com/course/tableau10/,bussiness,[Data Scientist,609473 Reviews,2277515 Students,339236 students,104 Courses,4.5 Instructor Rating,https://img-c.udemycdn.com/course/240x135/937678_abd2_3.jpg,4.6,(88565 ratings),Kirill Eremenko,Last updated 1/2023]
2,PMP Exam Prep Seminar - PMBOK Guide 6,PMP Exam Prep Seminar - Earn 35 PDUs by completing the entire PMP course,157957,4.6,53858,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,bussiness,[PMP PMI-ACP Project+ ITIL PSM I CTT+,213474 Reviews,682347 Students,291967 students,44 Courses,4.6 Instructor Rating,https://img-b.udemycdn.com/course/240x135/1361790_2eb7_2.jpg,4.6,(92161 ratings),Joseph Phillips,Last updated 1/2023]
3,The Complete Financial Analyst Course 2020,"Excel, Accounting, Financial Statement Analysis, Business Analysis, Financial Math, PowerPoint: Everything is Included!",249097,4.5,47415,https://www.udemy.com/course/the-complete-financial-analyst-course/,bussiness,[Creating opportunities for Data Science and Finance students,652690 Reviews,2270687 Students,375991 students,91 Courses,4.5 Instructor Rating,https://img-b.udemycdn.com/course/240x135/648826_f0e5_4.jpg,4.6,4.6,365 Careers,Last updated 1/2023]
4,An Entire MBA in 1 Course:Award Winning Business School Prof,** #1 Best Selling Business Course! ** Everything You Need to Know About Business from Start-up to IPO,376913,4.5,42101,https://www.udemy.com/course/an-entire-mba-in-1-courseaward-winning-business-school-prof/,bussiness,[CEO of Haroun MBA Degree Program®,208696 Reviews,1312283 Students,459986 students,70 Courses,4.5 Instructor Rating,https://img-c.udemycdn.com/course/240x135/637930_9a22_19.jpg,4.5,4.5,Chris Haroun,Last updated 2/2023]
5,Microsoft Power BI - A Complete Introduction [2020 EDITION],"Learn how to use Microsoft's Power BI Tools, including Power BI Desktop, Power BI Pro (Service) and PowerBI Developer",126880,4.6,38771,https://www.udemy.com/course/powerbi-complete-introduction/,bussiness,[Professional Business Analyst and Instructor,906256 Reviews,2405327 Students,254477 students,58 Courses,4.6 Instructor Rating,https://img-c.udemycdn.com/course/240x135/1208634_cd50_2.jpg,4.6,(64066 ratings),Manuel Lorenz,Last updated 11/2022]
6,Agile Crash Course: Agile Project Management; Agile Delivery,Get Agile Certified & Learn about the key and most important concepts and tools of Agile Project Management (Scrum),98700,4.3,31276,https://www.udemy.com/course/agile-crash-course/,bussiness,[Serial entrepreneur techie life hacker expert PM & MBA,120540 Reviews,449464 Students,220113 students,44 Courses,4.4 Instructor Rating,https://img-c.udemycdn.com/course/240x135/864146_dbd7_10.jpg,4.5,(75805 ratings),Mauricio Rubio - Agile Guru & Founder of AgileKB | AgileLee & Ureducation,Last updated 2/2023]
7,Beginner to Pro in Excel: Financial Modeling and Valuation,Financial Modeling in Excel that would allow you to walk into a job and be a rockstar from day one!,128940,4.5,29111,https://www.udemy.com/course/beginner-to-pro-in-excel-financial-modeling-and-valuation/,bussiness,[Creating opportunities for Data Science and Finance students,652690 Reviews,2270687 Students,193315 students,91 Cour

In [0]:
from pyspark.sql.functions import udf,substring
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import length
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import StringType
import re
from pyspark.sql.functions import current_timestamp

# get_integers_before_space = udf(lambda s:r'\d+' if s else None, IntegerType())

# add new column with existing column
# result_df = result_df.withColumn('lens', length(result_df['written_rating']))

result_df= result_df.withColumn('written_rating', regexp_extract(result_df['written_rating'], '^\\D*(\\d+)', 1))


def collect_only_digits_before_space(value):
    import re
    pattern = r'^\D*(\d+)'
    match = re.search(pattern, str(value))
    if match:
        return match.group(1)
    else:
        return None
    

def extract_month_year(value):
    pattern = r'Last updated (\d{1,2}/\d{4})'
    match = re.search(pattern, str(value))
    if match:
        return match.group(1)
    else:
        return None


my_udf = udf(collect_only_digits_before_space, StringType())
extract_month_year_udf = udf(extract_month_year, StringType())

result_df = result_df.withColumn('total_students_author', my_udf(result_df['total_students_author']))
result_df = result_df.withColumn('latest_enroll', my_udf(result_df['latest_enroll']))
# latest_enroll
# result_df = result_df.withColumn('total_students', udf(my_udf, IntegerType())(result_df['total_students']).cast('integer'))
result_df = result_df.withColumn('total_courses', my_udf(result_df['total_courses']))
result_df = result_df.withColumn('instructor_rating', substring(result_df['instructor_rating'], 1, 4))
result_df = result_df.withColumn('written_course_rating', my_udf(result_df['written_course_rating']))
result_df = result_df.withColumn('last_modified', extract_month_year_udf('last_modified'))
result_df = result_df.withColumn("created_timestamp", current_timestamp())




In [0]:
silver_udemy_schema=[["index","int" ]
,["Title" ,"string" ]
,["Summary","string" ]
,["Enrollment","int" ]
,["Stars","double"]
,["Rating","int" ]
,["Link","string" ]
,["category","string" ]
,["instructor_tag","string" ]
,["written_rating","int" ]
,["total_students_author","int" ]
 ,["latest_enroll","int" ]                  
,["total_courses","int" ]
,["instructor_rating","double"]
,["img_url","string" ]
,["course_rating","double"]
,["written_course_rating","int" ]
,["author_name","string" ]
,["last_modified","string"]
,["created_timestamp","timestamp"]]

In [0]:
from pyspark.sql.functions import to_date,when,lpad
df_udemy_silver = result_df
for column, datatype in silver_udemy_schema:
  df_udemy_silver = df_udemy_silver.withColumn(column, col(column).cast(datatype))

df_udemy_silver = df_udemy_silver.withColumn("last_modified", lpad("last_modified", 7, "0"))

In [0]:
display(df_udemy_silver)

index,Title,Summary,Enrollment,Stars,Rating,Link,category,instructor_tag,written_rating,total_students_author,latest_enroll,total_courses,instructor_rating,img_url,course_rating,written_course_rating,author_name,last_modified,created_timestamp
0,The Complete SQL Bootcamp 2020: Go from Zero to Hero,Become an expert at SQL!,301243,4.7,79919,https://www.udemy.com/course/the-complete-sql-bootcamp/,bussiness,[Head of Data Science at Pierian Training,1013752,3269886,641558,60,4.6,https://img-b.udemycdn.com/course/240x135/762616_7693_3.jpg,4.7,164866,Jose Portilla,05/2022,2023-03-06T14:54:49.752+0000
1,Tableau 2020 A-Z: Hands-On Tableau Training for Data Science,Learn Tableau 2020 for data science step by step. Real-life data analytics exercises & quizzes included. Learn by doing!,211674,4.6,55582,https://www.udemy.com/course/tableau10/,bussiness,[Data Scientist,609473,2277515,339236,104,4.5,https://img-c.udemycdn.com/course/240x135/937678_abd2_3.jpg,4.6,88565,Kirill Eremenko,01/2023,2023-03-06T14:54:49.752+0000
2,PMP Exam Prep Seminar - PMBOK Guide 6,PMP Exam Prep Seminar - Earn 35 PDUs by completing the entire PMP course,157957,4.6,53858,https://www.udemy.com/course/pmp-pmbok6-35-pdus/,bussiness,[PMP PMI-ACP Project+ ITIL PSM I CTT+,213474,682347,291967,44,4.6,https://img-c.udemycdn.com/course/240x135/1361790_2eb7_2.jpg,4.6,92161,Joseph Phillips,01/2023,2023-03-06T14:54:49.752+0000
3,The Complete Financial Analyst Course 2020,"Excel, Accounting, Financial Statement Analysis, Business Analysis, Financial Math, PowerPoint: Everything is Included!",249097,4.5,47415,https://www.udemy.com/course/the-complete-financial-analyst-course/,bussiness,[Creating opportunities for Data Science and Finance students,652690,2270687,375991,91,4.5,https://img-c.udemycdn.com/course/240x135/648826_f0e5_4.jpg,4.6,4,365 Careers,01/2023,2023-03-06T14:54:49.752+0000
4,An Entire MBA in 1 Course:Award Winning Business School Prof,** #1 Best Selling Business Course! ** Everything You Need to Know About Business from Start-up to IPO,376913,4.5,42101,https://www.udemy.com/course/an-entire-mba-in-1-courseaward-winning-business-school-prof/,bussiness,[CEO of Haroun MBA Degree Program®,208696,1312283,459986,70,4.5,https://img-b.udemycdn.com/course/240x135/637930_9a22_19.jpg,4.5,4,Chris Haroun,02/2023,2023-03-06T14:54:49.752+0000
5,Microsoft Power BI - A Complete Introduction [2020 EDITION],"Learn how to use Microsoft's Power BI Tools, including Power BI Desktop, Power BI Pro (Service) and PowerBI Developer",126880,4.6,38771,https://www.udemy.com/course/powerbi-complete-introduction/,bussiness,[Professional Business Analyst and Instructor,906256,2405327,254477,58,4.6,https://img-c.udemycdn.com/course/240x135/1208634_cd50_2.jpg,4.6,64066,Manuel Lorenz,11/2022,2023-03-06T14:54:49.752+0000
6,Agile Crash Course: Agile Project Management; Agile Delivery,Get Agile Certified & Learn about the key and most important concepts and tools of Agile Project Management (Scrum),98700,4.3,31276,https://www.udemy.com/course/agile-crash-course/,bussiness,[Serial entrepreneur techie life hacker expert PM & MBA,120540,449464,220113,44,4.4,https://img-c.udemycdn.com/course/240x135/864146_dbd7_10.jpg,4.5,75805,Mauricio Rubio - Agile Guru & Founder of AgileKB | AgileLee & Ureducation,02/2023,2023-03-06T14:54:49.752+0000
7,Beginner to Pro in Excel: Financial Modeling and Valuation,Financial Modeling in Excel that would allow you to walk into a job and be a rockstar from day one!,128940,4.5,29111,https://www.udemy.com/course/beginner-to-pro-in-excel-financial-modeling-and-valuation/,bussiness,[Creating opportunities for Data Science and Finance students,652690,2270687,193315,91,4.5,https://img-b.udemycdn.com/course/240x135/321410_d9c5_4.jpg,4.5,4,365 Careers,02/2023,2023-03-06T14:54:49.752+0000
8,Become a Product Manager | Learn the Skills & Get the Job,"The most complete course available on Product Management. 13+ hours of videos, activities, interviews, & more",114114,4.5,27879,https://www

In [0]:
# movie_name = input("Enter the name of the movie: ") 
# try:
#     user_search = df_udemy_silver.filter(df_udemy_silver.Title == movie_name).select("Genre").collect()[0][0]`
#     genre =genre.split("/")    
#     related_courses = df_udemy_silver.filter(df_udemy_silver.Title == 
# except IndexError :
#     print('there is only movie related to that genre') 
# display(related_courses)

In [0]:
df_udemy_silver.write.format("parquet").mode("overwrite").saveAsTable("silver.udemy_all_string")
# df_udemy_silver = df_udemy_silver.withColumn("last_modified", to_date("last_modified", "MM/yyyy"))

In [0]:
df_udemy_silver = spark.read.table("silver.udemy_all_string")

In [0]:
display(df_udemy_silver)


index,Title,Summary,Enrollment,Stars,Rating,Link,category,instructor_tag,written_rating,total_students_author,latest_enroll,total_courses,instructor_rating,img_url,course_rating,written_course_rating,author_name,last_modified,created_timestamp
0,The Complete Digital Marketing Course - 12 Courses in 1,"Master Digital Marketing Strategy, Social Media Marketing, SEO, YouTube, Email, Facebook Marketing, Analytics & More!",494107,4.4,117014,https://www.udemy.com/course/learn-digital-marketing-course/,marketing,[Web Developer And Teacher,463464,2205928,737753,80,4.5,https://img-b.udemycdn.com/course/240x135/914296_3670_8.jpg,4.5,160871,Rob Percival,11/2022,2023-03-06T14:58:34.549+0000
1,Ultimate Google Ads Training 2020: Profit with Pay Per Click,Google Ads 2020: How our clients have transformed their sales using Google Ads & get your Google Ads certification!,159200,4.7,39979,https://www.udemy.com/course/the-ultimate-google-adwords-training-course/,marketing,[Certified Google AdWords Pro |Co-founder of AdVenture Media,63417,239054,207369,4,4.6,https://img-c.udemycdn.com/course/240x135/671544_9613_5.jpg,4.6,50092,Isaac Rudansky,10/2021,2023-03-06T14:58:34.549+0000
2,Instagram Marketing 2020: Complete Guide To Instagram Growth,"Attract Hyper-Targeted Instagram Followers, Convert Followers to Paying Customers, & Expand your Brand Using Instagram",128185,4.4,24520,https://www.udemy.com/course/instagram-marketing-for-small-businesses/,marketing,[Passionate Entrepreneur,40274,244423,182624,11,4.5,https://img-b.udemycdn.com/course/240x135/484606_5721_16.jpg,4.5,35007,Benjamin Wilson,10/2021,2023-03-06T14:58:34.549+0000
3,Facebook Ads & Facebook Marketing MASTERY 2020 | Coursenvy ®,"Facebook Marketing from beginner to advanced! Join 100,000+ students who MASTERED Facebook and are Facebook Ads experts!",157240,4.4,24351,https://www.udemy.com/course/facebook-ads-facebook-marketing-mastery-guide/,marketing,[COURSENVY - Facebook Ads SMM Expert Ad Agency Amazon FBA,67064,479282,208439,26,4.4,https://img-b.udemycdn.com/course/240x135/403314_b091_3.jpg,4.5,31870,COURSE ENVY,03/2023,2023-03-06T14:58:34.549+0000
4,Digital Marketing Masterclass - 23 Courses in 1,"Grow Your Business with Digital Marketing: Social Media Marketing, Facebook, Content, YouTube, Email Marketing, Websites",188603,4.4,14107,https://www.udemy.com/course/digital-marketing-masterclass/,marketing,[Top-Rated Instructor 2.5 Million+ Students,144932,776445,235908,72,4.6,https://img-c.udemycdn.com/course/240x135/1270870_8e95_14.jpg,4.5,20674,Phil Ebiner,12/2022,2023-03-06T14:58:34.549+0000
5,SQL for Data Analysis: Weekender Crash Course for Beginners,"Using MySQL but applicable to Oracle SQL, Microsoft SQL Server, and PostgreSQL. Taught by a Data Scientist and PM.",59684,4.3,11851,https://www.udemy.com/course/sql-for-newbs/,marketing,[A Course Series,14924,71332,71335,1,4.4,https://img-c.udemycdn.com/course/240x135/203556_5ff1_3.jpg,4.4,14924,A Course You'll Actually Finish,09/2022,2023-03-06T14:58:34.549+0000
6,Social Media Marketing MASTERY | Learn Ads on 10+ Platforms,"MASTER online marketing on Twitter, Pinterest, Instagram, YouTube, Facebook, Google and more ad platforms! Coursenvy ®",80997,4.4,11617,https://www.udemy.com/course/social-media-marketing-ads/,marketing,[COURSENVY - Facebook Ads SMM Expert Ad Agency Amazon FBA,67064,479282,123514,26,4.4,https://img-b.udemycdn.com/course/240x135/573984_ce2a_3.jpg,4.4,19583,COURSE ENVY,02/2023,2023-03-06T14:58:34.549+0000
7,Google Analytics Certification: Become Certified & Earn More,"Become Google Analytics Certified to Land a Job, Get Promoted or Start a New Career in Digital Marketing! 2020 Guide",75384,4.5,11342,https://www.udemy.com/course/google-analytics-certification/,marketing,[Join 1 Million Students In My Best Selling Courses!,235191,1312593,107144,16,4.5,https://img-c.udemycdn.com/course/240x135/328386_6809_11.jpg,4.4,16669,Daragh Walsh,07/2021,2023-03-06T14:58:34.549+0000
8,SEO 2020: Complete SEO Training + SEO for Word

In [0]:
df_udemy_silver=df_udemy_silver.filter(df_udemy_silver.written_course_rating>4)

In [0]:
df_udemy_silver.write.format("parquet").mode("overwrite").saveAsTable("silver.udemy2")

In [0]:
display(df_udemy_silver)

index,Title,Summary,Enrollment,Stars,Rating,Link,category,instructor_tag,written_rating,total_students_author,latest_enroll,total_courses,instructor_rating,img_url,course_rating,written_course_rating,author_name,last_modified,created_timestamp
0,The Complete Digital Marketing Course - 12 Courses in 1,"Master Digital Marketing Strategy, Social Media Marketing, SEO, YouTube, Email, Facebook Marketing, Analytics & More!",494107,4.4,117014,https://www.udemy.com/course/learn-digital-marketing-course/,marketing,[Web Developer And Teacher,463464,2205928,737753,80,4.5,https://img-b.udemycdn.com/course/240x135/914296_3670_8.jpg,4.5,160871,Rob Percival,11/2022,2023-03-06T14:58:34.549+0000
1,Ultimate Google Ads Training 2020: Profit with Pay Per Click,Google Ads 2020: How our clients have transformed their sales using Google Ads & get your Google Ads certification!,159200,4.7,39979,https://www.udemy.com/course/the-ultimate-google-adwords-training-course/,marketing,[Certified Google AdWords Pro |Co-founder of AdVenture Media,63417,239054,207369,4,4.6,https://img-c.udemycdn.com/course/240x135/671544_9613_5.jpg,4.6,50092,Isaac Rudansky,10/2021,2023-03-06T14:58:34.549+0000
2,Instagram Marketing 2020: Complete Guide To Instagram Growth,"Attract Hyper-Targeted Instagram Followers, Convert Followers to Paying Customers, & Expand your Brand Using Instagram",128185,4.4,24520,https://www.udemy.com/course/instagram-marketing-for-small-businesses/,marketing,[Passionate Entrepreneur,40274,244423,182624,11,4.5,https://img-b.udemycdn.com/course/240x135/484606_5721_16.jpg,4.5,35007,Benjamin Wilson,10/2021,2023-03-06T14:58:34.549+0000
3,Facebook Ads & Facebook Marketing MASTERY 2020 | Coursenvy ®,"Facebook Marketing from beginner to advanced! Join 100,000+ students who MASTERED Facebook and are Facebook Ads experts!",157240,4.4,24351,https://www.udemy.com/course/facebook-ads-facebook-marketing-mastery-guide/,marketing,[COURSENVY - Facebook Ads SMM Expert Ad Agency Amazon FBA,67064,479282,208439,26,4.4,https://img-b.udemycdn.com/course/240x135/403314_b091_3.jpg,4.5,31870,COURSE ENVY,03/2023,2023-03-06T14:58:34.549+0000
4,Digital Marketing Masterclass - 23 Courses in 1,"Grow Your Business with Digital Marketing: Social Media Marketing, Facebook, Content, YouTube, Email Marketing, Websites",188603,4.4,14107,https://www.udemy.com/course/digital-marketing-masterclass/,marketing,[Top-Rated Instructor 2.5 Million+ Students,144932,776445,235908,72,4.6,https://img-c.udemycdn.com/course/240x135/1270870_8e95_14.jpg,4.5,20674,Phil Ebiner,12/2022,2023-03-06T14:58:34.549+0000
5,SQL for Data Analysis: Weekender Crash Course for Beginners,"Using MySQL but applicable to Oracle SQL, Microsoft SQL Server, and PostgreSQL. Taught by a Data Scientist and PM.",59684,4.3,11851,https://www.udemy.com/course/sql-for-newbs/,marketing,[A Course Series,14924,71332,71335,1,4.4,https://img-c.udemycdn.com/course/240x135/203556_5ff1_3.jpg,4.4,14924,A Course You'll Actually Finish,09/2022,2023-03-06T14:58:34.549+0000
6,Social Media Marketing MASTERY | Learn Ads on 10+ Platforms,"MASTER online marketing on Twitter, Pinterest, Instagram, YouTube, Facebook, Google and more ad platforms! Coursenvy ®",80997,4.4,11617,https://www.udemy.com/course/social-media-marketing-ads/,marketing,[COURSENVY - Facebook Ads SMM Expert Ad Agency Amazon FBA,67064,479282,123514,26,4.4,https://img-b.udemycdn.com/course/240x135/573984_ce2a_3.jpg,4.4,19583,COURSE ENVY,02/2023,2023-03-06T14:58:34.549+0000
7,Google Analytics Certification: Become Certified & Earn More,"Become Google Analytics Certified to Land a Job, Get Promoted or Start a New Career in Digital Marketing! 2020 Guide",75384,4.5,11342,https://www.udemy.com/course/google-analytics-certification/,marketing,[Join 1 Million Students In My Best Selling Courses!,235191,1312593,107144,16,4.5,https://img-c.udemycdn.com/course/240x135/328386_6809_11.jpg,4.4,16669,Daragh Walsh,07/2021,2023-03-06T14:58:34.549+0000
9,The Complete Copywriting Course : Write to Sel